# Imports

In [1]:
import importlib
from functools import partial

from torch import nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import copy

from decision_learning.utils import handle_solver
from decision_learning.benchmarks.shortest_path_grid.data import genData
from decision_learning.modeling.loss import SPOPlus, get_loss_function
from decision_learning.modeling.models import LinearRegression
from decision_learning.modeling.val_metrics import decision_regret
from decision_learning.modeling.train import train, calc_test_regret, init_loss_data_pretraining, filter_kwargs

# Training Loop Overview
While user can run entire experiment pipeline using `decision_learning.modeling.pipeline`, sometimes they may want more detailed control over training data setup (compared to the default behavior in `decision_learning.modeling.pipeline.build_loss_data_dict`, or hyperparmeter searching (compared to the default grid search in the pipeline function `decision_learning.modeling.pipeline.lossfn_experiment_pipeline`). This may happen to a highly customized loss function or experiment process. In that case, we may still want a function to handle Pytorch training loop behavior and we can use the appropriate customized components without needing to delve into boilerplate Pytorch code. In that case, we can use the `decision_learning.modeling.train.train` function to handle pytorch training functionality

At a high level, the train function needs the following components:
- prediction model: predicting true costs
- optimization model: linear optimization model parameterized by cost/coefficient vector for objective function, and returns the corresponding solution, objective value.
- loss function: a callable nn.Module loss function that can be used for PyTorch autograd functionality
- structured training data inputs: dictionary mapping from keys to specific data (features, true costs, true obj, true sol), for training, validation, and test sets
- val metric: a callable function used during each epoch to evaluate model on the validation set
- misc training loop parameters: device, number epochs, optimizer, learning rate, etc.

## Optimization Model (linear)

Decision-aware/focused problems require an optimization model to actually solve the decision problem. Since each decision problem is unique in terms of the modeling and solving, the user is expected to provide the optimization model function/object, which is treated like a black-box by the `pipeline`,`train`, and loss/regret functions in the code base. It could be Gurobi, Pyomo, or any user custom solver. However, to play nicely with the rest of the package, it must do the following:

- Input Argument when called:
    - costs: vector of objective function coefficients. Expected to be numpy np.ndarray or torch.tensor
- Returns 2 objects:
    - sols: solution to optimization model given the input costs. Expected to be numpy np.ndarray or torch.tensor
    - obj: objective value to optimization model given the input costs. Expected to be numpy np.ndarray or torch.tensor
 
The return objects of optimal solution and objective are generally returned as any solver, and any linear program needs its objective function to be parameterized by a vector of cost/coefficients.


### Example Solver/Optimization Model
- Below, `shortest_path_solver` is a custom user optimization model specified in the form of a callable function, and its first input argument is the vector of costs. The rest of the input arguments size, sens, need to be pre-set before being passed to `pipeline`, `train`, or any loss function. This can be accomplished using the `partial` python function (see example below). The exact implementation is not important but mainly that it:
    - accepts a costs vector input
    - returns solution (sol) and objective value (obj) for the input cost vector
- Note that `shortest_path_solver` also has two returns: sol, obj

In [2]:
from decision_learning.benchmarks.shortest_path_grid.oracle import opt_oracle as shortest_path_solver


### Presetting non-cost input arguments of `shortest_path_solver`

In [3]:
# ------------optimization model------------
optmodel = partial(handle_solver, optmodel=shortest_path_solver, detach_tensor=False, solver_batch_solve=True)

## Data Generation Setup
Any decision-aware/focused problem will of course need data inputs. The example below uses a pre-implemented synthetic data generator provided in the package found within 
`decision_learning.benchmarks.shortest_path_grid.data` to generate shortest path problem and can be generated by calling the `genData` function

### Specific parameters to set up data generation
This data setup, and the synthetic data generation is in line with the paper https://arxiv.org/pdf/2402.03256 

### Create Experiments Grid
This shortest path experiment has two important settings:
- number of samples: less samples means higher error/more noise, more samples means lower error/less noise
- epsilon: noise level on edge costs, can be uniformly distributed multiplicative noise, or normally distributed additive noise
- This example below creates 100 trials for 8 different settings

In [4]:
# control the randomization seeding for pytorch
torch.manual_seed(105)
indices_arr = torch.randperm(100000)
indices_arr_test = torch.randperm(100000)

n_arr = [200, 400, 800, 1600] # array of number of samples for an experiment
ep_arr = ['unif', 'normal'] # noise type
trials = 100 # number of trials per setting

# create an array where each item is [number of samples, noise type, trial number] representing an experiment run
exp_arr = []
for n in n_arr:
    for ep in ep_arr:
        for t in range(trials):
            exp_arr.append([n, ep, t]) # add current [number of samples, noise type, trial number] experiment run setting

In [5]:
# setup
sim = 0 # simulation trial number, only show one experiment run for demonstration purposes
exp = exp_arr[sim] # current experiment
num_data = exp[0]  # number of training data
ep_type = exp[1] # noise type of current experiment
trial = exp[2] # trial number of current experiment

# shortest path problem data generation parameters - https://arxiv.org/pdf/2402.03256
grid = (5, 5)  # grid size
num_feat = 5  # size of feature
deg = 6  # polynomial degree in edge cost function
e = .3  # noise width/amount of noise

# path planting for shortest path example - see page 9, subsection "Harder Example with Planted Arcs" in section 4.2 of paper https://arxiv.org/pdf/2402.03256
planted_good_pwl_params = {'slope0':0, # slope of first segment of piecewise linear cost function for "good" edge cost planted
                    'int0':2, # intercept of first segment of piecewise linear cost function for "good" edge cost planted
                    'slope1':0, # slope of second segment of piecewise linear cost function for "good" edge cost planted
                    'int1':2} # intercept of second segment of piecewise linear cost function for "good" edge cost planted
planted_bad_pwl_params = {'slope0':4, # slope of first segment of piecewise linear cost function for "bad" edge cost planted
                    'int0':0, # intercept of first segment of piecewise linear cost function for "bad" edge cost planted
                    'slope1':0, # slope of second segment of piecewise linear cost function for "bad" edge cost planted
                    'int1':2.2} # intercept of second segment of piecewise linear cost function for "bad" edge cost planted
plant_edge = True # to plant edges in shortest path experiment or not

print(f'current experiment setting: number of data points {num_data}, epsilon type {ep_type}, trial number {trial}')

current experiment setting: number of data points 200, epsilon type unif, trial number 0


### Calling `genData` from `decision_learning.benchmarks.shortest_path_grid.data`

In [6]:
# ------------DATA------------
# training data
generated_data = genData(num_data=num_data+200, # number of data points to generate for training set
        num_features=num_feat, # number of features 
        grid=grid, # grid shape
        deg=deg, # polynomial degree
        noise_type=ep_type, # epsilon noise type
        noise_width=e, # amount of noise
        seed=indices_arr[trial], # seed the randomness
        plant_edges=plant_edge, # to plant edges or not
        planted_good_pwl_params=planted_good_pwl_params, # cost function for good edges
        planted_bad_pwl_params=planted_bad_pwl_params) # cost function for bad edges

# testing data
generated_data_test = genData(num_data=10000, # number of data points to generate for test set
        num_features=num_feat, # number of features 
        grid=grid,  # grid shape
        deg=deg,  # polynomial degree
        noise_type=ep_type,  # epsilon noise type
        noise_width=e, # amount of noise
        seed=indices_arr_test[trial],      # seed the randomness
        plant_edges=plant_edge, # to plant edges or not
        planted_good_pwl_params=planted_good_pwl_params, # cost function for good edges
        planted_bad_pwl_params=planted_bad_pwl_params) # cost function for bad edges

### Split Data into train, val and create structured data inputs
For many decision aware loss function experiment processes, we need the following four:
- X: features
- true_cost: true cost associated with each X
- true_sol: true solution of LP given true_cost
- true_obj: true objective of LP given true_cost

In this case, we will need to get the `true_sol` and `true_obj` for each data sample/`true_cost` vector by plugging into our optimization solver. The code block below shows this.

Furthermore, this package's training loop, in order to flexibly handle different loss function signatures and behavior, requires data to be organized as dictionaries mapping key->data where the keys correspond to named arguments in the loss function. This way, the training loop can flexibly inject or remove the correct named arguments to each loss function

In [23]:
# training data - get true_sol, true_obj
sol, obj = optmodel(generated_data['cost'],
                instance_kwargs={'size': 5}) # plug into solver
                                                                                                                        
# get structured data in the form of dictionary
final_data = {'X':generated_data['feat'],
              'true_cost':generated_data['cost'],
              'true_sol':sol,
              'true_obj':obj,
              'instance_kwargs': {'size': np.zeros(len(generated_data['cost'])) + 5}}

# ------------------TRAIN/VAL SPLIT--------------------
train_dict, val_dict = train_val_spl(train_d=final_data, val_split_params={'test_size':200,
                                                                'random_state':42})


# test data - get true_sol, true_obj and structured data form
sol_test, obj_test = optmodel(generated_data_test['cost_true'],
                            instance_kwargs={'size': 5}) # plug into solver
final_data_test = {'X':generated_data_test['feat'],
              'true_cost':generated_data_test['cost_true'],
              'true_sol':sol_test,
              'true_obj':obj_test,
              'instance_kwargs': {'size': np.zeros(len(generated_data_test['cost'])) + 5}}

# training, validation data
train_data_dict = train_dict
val_data_dict = val_dict

/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_7610/1157061115.py:27: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  next_V_val_l[:, :num_nodes_next - 1] = V_1 + l_costs
/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_7610/1157061115.py:28: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  next_V_val_r[:, 1:num_nodes_next] = V_1 + r_costs
/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_7610/1157061115.py:30: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  next_V_val_l = V_1[:, :num_nodes_next] + l_costs
/var/folders/k9/q1b_kr4s353_ckw475n0fxw40000gn/T/ipykernel_7610/1157061115.py:31: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the f

In [22]:
import decision_learning.modeling.pipeline 
importlib.reload(decision_learning.modeling.pipeline )
from decision_learning.modeling.pipeline import train_val_spl

In [15]:
final_data['instance_kwargs'] = {'size': np.zeros(len(generated_data['feat'])) + 5}

# ------------------TRAIN/VAL SPLIT--------------------
train_dict_new, val_dict_new = train_val_spl(train_d=final_data, val_split_params={'test_size':200,
                                                                                'random_state':42})

In [20]:
for k in train_dict_new:
    print(k)
    
    if k == 'instance_kwargs':
        for sol_k in train_dict_new[k]:
            print(k, sol_k)
            print(np.array_equal(train_dict_new[k][sol_k], train_dict[k][sol_k]))
    else:
        print(np.array_equal(train_dict_new[k], train_dict[k]))

X
True
true_cost
True
true_sol
True
true_obj
True
instance_kwargs
instance_kwargs size
True


## Prediction Model
- Any decision-aware/focused problem will of course need prediction model to predict the cost/coefficient vector given contextual input/features. This example uses a simple `LinearRegression` object implemented within `decision_learning.modeling.models`. 
- The package expects the prediction model to be a PyTorch model since PyTorch offers convenient autograd functionality/allows user to specify custom losses/backwards passes that are found within many decision-aware/focused works.

In [24]:
# ------------prediction model------------
pred_model = LinearRegression(input_dim=generated_data['feat'].shape[1],
                 output_dim=generated_data['cost'].shape[1])

### Loss Function
Below, we use the preimplemented `SPOPlus` loss from `decision_learning.modeling.loss`, which requires an optimization model input for solving for sol,obj under current predicted costs for loss, backpropogation each epoch

In [25]:
# loss function
loss_fn = SPOPlus(optmodel=optmodel)

In [26]:
metrics, trained_model = train(pred_model=copy.deepcopy(pred_model), # prediction model
                optmodel=optmodel, # optimization model
                loss_fn=loss_fn, # loss function
                train_data_dict=train_data_dict, # training data dictionary
                val_data_dict=val_data_dict, # validation data dictionary
                test_data_dict=final_data_test, # test data dictionary
                dataloader_params={'batch_size':200, 'shuffle':True}, # pytorch dataloader configuration
                num_epochs=100, # number of epochs to train for
                lr=0.01, # learning rate
                scheduler_params=None, # learning rate scheduler - example: {'step_size': 10, 'gamma': 0.1}, None means no scheduler
                minimization=True, # minimization problem
                verbose=True) # output training loop details - better to train off sometimes if no console output wanted for many experiments

2025-02-04 10:16:37,612 - decision_learning.modeling.train - INFO - Training on device: cpu
Training Loader: Epoch 1/100: 100%|██████████| 1/1 [00:00<00:00, 22.06it/s]
2025-02-04 10:16:37,674 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 17.651004791259766, val_metric: 0.508552379022715, test_regret: 0.46433861551426975
Training Loader: Epoch 2/100: 100%|██████████| 1/1 [00:00<00:00, 366.15it/s]
2025-02-04 10:16:37,685 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 16.961898803710938, val_metric: 0.49043997563631775, test_regret: 0.4475846525963112
Training Loader: Epoch 3/100: 100%|██████████| 1/1 [00:00<00:00, 394.80it/s]
2025-02-04 10:16:37,694 - decision_learning.modeling.train - INFO - epoch: 3, train_loss: 16.293970108032227, val_metric: 0.4665094928364167, test_regret: 0.430050970873209
Training Loader: Epoch 4/100: 100%|██████████| 1/1 [00:00<00:00, 364.44it/s]
2025-02-04 10:16:37,703 - decision_learning.modeling.train - INFO - epoch: 4, t

In [27]:
metrics

,epoch,train_loss,val_metric,test_regret
0,0,17.651005,0.508552,0.464339
1,1,16.961899,0.490440,0.447585
2,2,16.293970,0.466509,0.430051
3,3,15.672397,0.462529,0.407156
4,4,15.074890,0.433064,0.384136
...,...,...,...,...
95,95,4.232066,0.096306,0.045160
96,96,4.225177,0.093261,0.045188
97,97,4.218335,0.090846,0.044655
98,98,4.211622,0.090218,0.044606


# Evaluation Regret
After training model, we may want to evaluate the model on a separate performance metric like normalized regret on a test dataset. While this is automatically done as part of the `train` function, here we explicitly show it for clarity

In [96]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test, #final_data_te,
                               optmodel=optmodel)
print(test_regret)

0.04437179557780305


# Further Examples
Since we may want to try different loss functions for decision aware problems, below are two more examples of preimplemented loss functions. The key is to ensure the training data dictionary named keys match the named arguments of the specific loss function. (Note we don't need to do this for val, test data dictionaries since those are not passed to loss function, but rather the validation metric)

## MSE

In [98]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(handle_solver, optmodel=shortest_path_solver, detach_tensor=False, solver_batch_solve=True)

# loss function
loss_fn = nn.MSELoss()

# training, validation data - need to modify training data dict to match inputs to loss function
train_data_dict = train_dict
train_data_dict.update({'target':train_dict['true_cost']}) # extra input key needed for loss function

val_data_dict = val_dict

In [99]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

2025-02-04 09:49:25,476 - decision_learning.modeling.train - INFO - Training on device: cpu
Training Loader: Epoch 1/100: 100%|██████████| 7/7 [00:00<00:00, 925.43it/s]
2025-02-04 09:49:25,490 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 5.732713460922241, val_metric: 0.3064160193437469, test_regret: nan
Training Loader: Epoch 2/100: 100%|██████████| 7/7 [00:00<00:00, 1785.79it/s]
2025-02-04 09:49:25,497 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 1.8749064377376012, val_metric: 0.23056091634093134, test_regret: nan
Training Loader: Epoch 3/100: 100%|██████████| 7/7 [00:00<00:00, 1899.10it/s]
2025-02-04 09:49:25,503 - decision_learning.modeling.train - INFO - epoch: 3, train_loss: 1.2857146774019514, val_metric: 0.185641400976215, test_regret: nan
Training Loader: Epoch 4/100: 100%|██████████| 7/7 [00:00<00:00, 2287.33it/s]
2025-02-04 09:49:25,509 - decision_learning.modeling.train - INFO - epoch: 4, train_loss: 1.3041847688811166, val_metric:

In [100]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.05796618960108441


## Cosine Loss

In [43]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(handle_solver, optmodel=shortest_path_solver, detach_tensor=False, solver_batch_solve=True)

# loss function
loss_fn = nn.CosineEmbeddingLoss()

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [44]:
print(train_data_dict)
print(val_data_dict)
print(final_data_test)

{'X': array([[-0.08355108,  0.45676513,  0.85631996, -0.15565138,  0.64988254,
         1.04705362],
       [ 1.75985576,  1.07878068, -0.36419321,  0.80513901,  2.11095927,
         0.49760102],
       [ 0.4820955 ,  0.12926345,  0.21688263, -1.22067403,  1.32201381,
         0.024743  ],
       ...,
       [ 0.46099305, -1.82424305, -0.44784871,  1.11174056, -0.1367769 ,
         1.23607579],
       [ 1.20149625, -0.48731733,  0.19152531, -0.39790225,  1.03802769,
         0.59617548],
       [ 1.44531054,  0.03414607,  2.55435546,  1.58195344, -1.02250734,
         0.8813497 ]], shape=(200, 6)), 'true_cost': array([[ 2.43645566,  1.5301999 ,  2.89246724, ...,  3.70157463,
         2.7937961 ,  1.75843334],
       [ 1.42097924,  2.57926467,  3.34271283, ..., 11.09371921,
         1.67972199,  2.56409644],
       [ 0.12156991,  1.44871366,  2.27142999, ...,  3.29269509,
         0.09236643,  1.73918022],
       ...,
       [ 2.33922265,  1.50827519,  2.59359074, ...,  2.28553073,
    

In [45]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                test_data_dict=final_data_test,
                num_epochs=100,
                lr=0.1, #scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

2025-02-04 11:11:28,776 - decision_learning.modeling.train - INFO - Training on device: cpu
Training Loader: Epoch 1/100: 100%|██████████| 7/7 [00:00<00:00, 1082.92it/s]
2025-02-04 11:11:28,798 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.3236325542841639, val_metric: 0.25299825877629867, test_regret: 0.1858860556500013
Training Loader: Epoch 2/100: 100%|██████████| 7/7 [00:00<00:00, 1051.24it/s]
2025-02-04 11:11:28,819 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.056552548493657796, val_metric: 0.16912549811790784, test_regret: 0.11657044179839648
Training Loader: Epoch 3/100: 100%|██████████| 7/7 [00:00<00:00, 1654.46it/s]
2025-02-04 11:11:28,831 - decision_learning.modeling.train - INFO - epoch: 3, train_loss: 0.03889621155602591, val_metric: 0.143314937203472, test_regret: 0.083318278891843
Training Loader: Epoch 4/100: 100%|██████████| 7/7 [00:00<00:00, 1608.07it/s]
2025-02-04 11:11:28,843 - decision_learning.modeling.train - INFO - ep

In [46]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.028437631999235918


In [103]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.03412545521451273


## CosineSurrogate

In [104]:
import importlib

import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
from decision_learning.modeling.loss import get_loss_function, CosineSurrogateDotProdMSE, CosineSurrogateDotProdVecMag

In [105]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(handle_solver, optmodel=shortest_path_solver, detach_tensor=False, solver_batch_solve=True)

# loss function
loss_fn = CosineSurrogateDotProdVecMag(alpha=5)

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [106]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

2025-02-04 09:50:02,886 - decision_learning.modeling.train - INFO - Training on device: cpu
Training Loader: Epoch 1/100: 100%|██████████| 7/7 [00:00<00:00, 951.77it/s]
2025-02-04 09:50:02,899 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 18.34548463140215, val_metric: 0.25724507298517296, test_regret: nan
Training Loader: Epoch 2/100: 100%|██████████| 7/7 [00:00<00:00, 1565.37it/s]
2025-02-04 09:50:02,907 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: -0.11923847879682269, val_metric: 0.19468765971938656, test_regret: nan
Training Loader: Epoch 3/100: 100%|██████████| 7/7 [00:00<00:00, 1468.74it/s]
2025-02-04 09:50:02,915 - decision_learning.modeling.train - INFO - epoch: 3, train_loss: -8.787602356501989, val_metric: 0.20635544991957974, test_regret: nan
Training Loader: Epoch 4/100: 100%|██████████| 7/7 [00:00<00:00, 1119.29it/s]
2025-02-04 09:50:02,924 - decision_learning.modeling.train - INFO - epoch: 4, train_loss: -12.032219750540596, val_m

In [107]:
test_regret = calc_test_regret(pred_model=trained_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.04959477865703449
